In [1]:
# Module Import

from __future__ import print_function
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt

In [2]:
## Google Drive Mount 

from google.colab import drive
drive.mount('/content/hw_drive')


## Trainig Setup

batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"="*44}')


## MNIST dataset Download 
train_dataset = datasets.MNIST(root='/content/hw_drive/My Drive/mai1/mnist_data/', 
                              train=True, 
                              transform=transforms.ToTensor(),
                              download=True)
test_dataset = datasets.MNIST(root='/content/hw_drive/My Drive/mai1/mnist_data/',
                             train=False,
                             transform=transforms.ToTensor())

## Data loader
train_loader = data.DataLoader(dataset=train_dataset,
                               batch_size=batch_size,
                               shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset,
                              batch_size=batch_size,
                              shuffle=False)

Mounted at /content/hw_drive
Training MNIST Model on cuda


In [3]:
## Network Architecture Definition

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.l1 = nn.Linear(784, 520)
    self.l2 = nn.Linear(520, 320)
    self.l3 = nn.Linear(320, 240)
    self.l4 = nn.Linear(240, 120)
    self.l5 = nn.Linear(120, 10)

  def forward(self, x):
    x = x.view(-1, 784)   #Flatten the data ( n, 1, 28, 28 ) -> (n, 784)
    x = F.relu(self.l1(x))
    x = F.relu(self.l2(x))
    x = F.relu(self.l3(x))
    x = F.relu(self.l4(x))
    return self.l5(x)

## Network Load

model = Net()
model.to(device)

Net(
  (l1): Linear(in_features=784, out_features=520, bias=True)
  (l2): Linear(in_features=520, out_features=320, bias=True)
  (l3): Linear(in_features=320, out_features=240, bias=True)
  (l4): Linear(in_features=240, out_features=120, bias=True)
  (l5): Linear(in_features=120, out_features=10, bias=True)
)

In [4]:
## Training Setup

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


In [5]:
## Train Function 

def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 10 == 0:
      print('Train Epoch : {} | Batch Status : {}/{} ({:.0f}%) | Loss : {:.6f}'.format(
          epoch, batch_idx*len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))


## Test Function

def test():
  model.eval()
  test_loss = 0
  correct = 0
  for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    output = model(data)

    # sum up batch loss
    test_loss += criterion(output, target).item()

    # get the index of the max
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

  test_loss /= len(test_loader.dataset)
  print(f'==================\nTest set: Average loss : {test_loss:.4f}, Accuracy : {correct}/{len(test_loader.dataset)}'
        f'({100. * correct / len(test_loader.dataset):.0f}%)')

In [6]:
## Main

if __name__ == '__main__':
 
  since = time.time()
  for epoch in range(1, 10):
    epoch_start = time.time()
    train(epoch)
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Training time: {m:.0f}m {s:.0f}s')
    
    test()
    m, s = divmod(time.time() - epoch_start, 60)
    print(f'Tesing time: {m:.0f}m {s:.0f}s')

  m, s = divmod(time.time() - epoch_start, 60)
  print(f'Total time : {m:.0f}m {s: .0f}s \nModel was trained on {device}!')

Train Epoch : 1 | Batch Status : 0/60000 (0%) | Loss : 2.303060
Train Epoch : 1 | Batch Status : 640/60000 (1%) | Loss : 2.302449
Train Epoch : 1 | Batch Status : 1280/60000 (2%) | Loss : 2.299678
Train Epoch : 1 | Batch Status : 1920/60000 (3%) | Loss : 2.299911
Train Epoch : 1 | Batch Status : 2560/60000 (4%) | Loss : 2.301847
Train Epoch : 1 | Batch Status : 3200/60000 (5%) | Loss : 2.297141
Train Epoch : 1 | Batch Status : 3840/60000 (6%) | Loss : 2.299596
Train Epoch : 1 | Batch Status : 4480/60000 (7%) | Loss : 2.299692
Train Epoch : 1 | Batch Status : 5120/60000 (9%) | Loss : 2.304871
Train Epoch : 1 | Batch Status : 5760/60000 (10%) | Loss : 2.301354
Train Epoch : 1 | Batch Status : 6400/60000 (11%) | Loss : 2.300351
Train Epoch : 1 | Batch Status : 7040/60000 (12%) | Loss : 2.297032
Train Epoch : 1 | Batch Status : 7680/60000 (13%) | Loss : 2.298703
Train Epoch : 1 | Batch Status : 8320/60000 (14%) | Loss : 2.294399
Train Epoch : 1 | Batch Status : 8960/60000 (15%) | Loss : 2.